In [5]:
import pandas as pd
import numpy as np
from datetime import timedelta
import os

# ==========================================
# CONFIGURATION & GROUND TRUTH
# ==========================================

# Timestamps for Data Exfiltration Scenarios (1, 2, 4, 5)
# Note: User PLJ1771 (Scenario 3 - IT Sabotage/Keylogger) is EXCLUDED from 
# the positive class (1) because it is not data exfiltration.
MALICIOUS_WINDOWS = {
    'ACM2278': ('2010-08-18 21:47:42', '2010-08-24 03:48:51'), # Scn 1: Wikileaks Upload
    'CMP2946': ('2011-02-07 12:28:06', '2011-03-04 12:30:25'), # Scn 2: Data Theft (USB)
    'CDE1846': ('2011-02-21 11:43:39', '2011-04-25 17:55:00'), # Scn 4: Email to Home
    'MBG3183': ('2010-10-12 13:21:59', '2010-10-12 13:22:56'), # Scn 5: Dropbox Upload
}

# Define the input file names (Adjust paths if your files are in a subfolder)
FILES = {
    'logon': '../Dataset/r6.2/logon.csv',
    'device': '../Dataset/r6.2/device.csv',
    'http': '../Dataset/r6.2/http.csv',
    'email': '../Dataset/r6.2/email.csv',
    'file': '../Dataset/r6.2/file.csv'
}

In [6]:
def load_and_parse(filepath, date_col='date'):
    """
    Helper function to load CSV and parse dates efficiently.
    Handles common encoding issues in CERT datasets.
    """
    print(f"Loading {filepath}...")
    if not os.path.exists(filepath):
        print(f"[!] Error: File not found {filepath}")
        return pd.DataFrame()
        
    try:
        # r6.2 often uses ISO-8859-1 encoding rather than utf-8
        df = pd.read_csv(filepath, encoding='ISO-8859-1') 
        # Coerce errors to NaT to prevent crashing on bad date formats
        df[date_col] = pd.to_datetime(df[date_col], format='%m/%d/%Y %H:%M:%S', errors='coerce')
        # Remove rows where date parsing failed
        return df.dropna(subset=[date_col]) 
    except Exception as e:
        print(f"[!] Error loading {filepath}: {e}")
        return pd.DataFrame()

In [7]:
# Load all log files
logon_df = load_and_parse(FILES['logon'])
device_df = load_and_parse(FILES['device'])
http_df = load_and_parse(FILES['http'])
email_df = load_and_parse(FILES['email'])
file_df = load_and_parse(FILES['file'])

print("All datasets loaded successfully.")

Loading ../Dataset/r6.2/logon.csv...
Loading ../Dataset/r6.2/device.csv...
Loading ../Dataset/r6.2/http.csv...
Loading ../Dataset/r6.2/email.csv...
Loading ../Dataset/r6.2/file.csv...
All datasets loaded successfully.


In [9]:
# Initialize the sessions list
sessions = []

print("Creating sessions from logon data...")

# Sort by User and Date to ensure chronological order
logon_df = logon_df.sort_values(by=['user', 'date'])

# Group by user to process distinct timelines
for user, user_logs in logon_df.groupby('user'):
    user_logs = user_logs.reset_index(drop=True)
    
    i = 0
    while i < len(user_logs):
        row = user_logs.iloc[i]
        
        # We start a session when we see a 'Logon'
        if row['activity'] == 'Logon':
            session_start = row['date']
            pc = row['pc']
            session_id = row['id']
            
            # Look ahead for the corresponding Logoff on the SAME PC
            session_end = None
            j = i + 1
            while j < len(user_logs):
                next_row = user_logs.iloc[j]
                
                if next_row['user'] == user and next_row['pc'] == pc:
                    if next_row['activity'] == 'Logoff':
                        session_end = next_row['date']
                        i = j # Advance outer loop to skip this logoff
                        break
                    elif next_row['activity'] == 'Logon':
                        # Found a new logon before a logoff (user didn't log out)
                        break
                j += 1
            
            # Handling Missing Logoffs: Cap session at 12 hours max if no logoff found
            if session_end is None:
                session_end = session_start + timedelta(hours=12)
            
            sessions.append({
                'id': session_id,
                'user': user,
                'pc': pc,
                'session_start': session_start,
                'session_end': session_end,
                'duration_sec': (session_end - session_start).total_seconds()
            })
        i += 1

sessions_df = pd.DataFrame(sessions)
print(f"Generated {len(sessions_df)} unique sessions.")

Creating sessions from logon data...
Generated 1881109 unique sessions.


In [13]:
# Initialize list to store final dataset rows
final_rows = []

print("Aggregating activity logs into sessions (this process takes time)...")

# Process per user to speed up filtering
for user, user_sessions in sessions_df.groupby('user'):
    
    # 1. Pre-filter other dataframes for this user (Optimization)
    u_email = email_df[email_df['user'] == user] if not email_df.empty else pd.DataFrame()
    u_http = http_df[http_df['user'] == user] if not http_df.empty else pd.DataFrame()
    u_device = device_df[device_df['user'] == user] if not device_df.empty else pd.DataFrame()
    u_file = file_df[file_df['user'] == user] if not file_df.empty else pd.DataFrame()

    for _, session in user_sessions.iterrows():
        start = session['session_start']
        end = session['session_end']
        
        # --- EMAIL FEATURES ---
        # Get emails sent during this session
        sess_emails = u_email[(u_email['date'] >= start) & (u_email['date'] <= end)]
        email_count = len(sess_emails)
        
        # Concatenate email content for the Autoencoder text field
        email_text_list = []
        for _, r in sess_emails.iterrows():
            to_val = str(r['to']) if pd.notna(r.get('to')) else ""
            content_val = str(r['content']) if pd.notna(r.get('content')) else ""
            email_text_list.append(f"TO: {to_val} BODY: {content_val}")
        
        email_content = " | ".join(email_text_list)

        # --- HTTP FEATURES ---
        sess_http = u_http[(u_http['date'] >= start) & (u_http['date'] <= end)]
        http_count = len(sess_http)
        
        http_text_list = []
        for _, r in sess_http.iterrows():
            url_val = str(r['url']) if pd.notna(r.get('url')) else ""
            content_val = str(r['content']) if pd.notna(r.get('content')) else ""
            http_text_list.append(f"{url_val} {content_val}")
        
        http_content = " | ".join(http_text_list)
        
        # --- DEVICE/FILE FEATURES ---
        sess_device = u_device[(u_device['date'] >= start) & (u_device['date'] <= end)]
        sess_files = u_file[(u_file['date'] >= start) & (u_file['date'] <= end)]
        
        device_count = len(sess_device)
        file_count = len(sess_files)
        
        # Calculate file copies to USB (Key indicator for Exfiltration)
        files_copied_to_usb = 0
        if 'to_removable_media' in sess_files.columns:
             files_copied_to_usb = len(sess_files[sess_files['to_removable_media'] == True])
        elif 'filename' in sess_files.columns:
            # Fallback: check if file path starts with removable drive letter (often R:)
            files_copied_to_usb = len(sess_files[sess_files['filename'].str.startswith('R:', na=False)])

        # --- LABELING LOGIC (FIXED) ---
        label = 0
        if user in MALICIOUS_WINDOWS:
            # FIX: Explicitly select index 0 for start and 1 for end
            mal_start = pd.to_datetime(MALICIOUS_WINDOWS[user][0])
            mal_end = pd.to_datetime(MALICIOUS_WINDOWS[user][1])
            
            # Check for time overlap
            if (start <= mal_end) and (end >= mal_start):
                label = 1

        # --- ROW CONSTRUCTION ---
        final_rows.append({
            'id': session['id'],
            'user': user,
            'session_start': start,
            'session_end': end,
            'duration': session['duration_sec'],
            'logon_activity': 1, 
            'email_activity': email_count,
            'email_content': email_content, 
            'http_activity': http_count,
            'http_content': http_content,
            'device_activity': device_count,
            'file_activity': file_count,
            'files_copied_to_usb': files_copied_to_usb,
            'label': label
        })

print("Aggregation complete.")

Aggregating activity logs into sessions (this process takes time)...


MemoryError: Unable to allocate 112. MiB for an array with shape (117025216,) and data type bool

In [ ]:
# Convert to DataFrame
df_final = pd.DataFrame(final_rows)

# Save
output_filename = "../CombinedDataset/cert_r6.2_session_dataset.csv"
df_final.to_csv(output_filename, index=False)

print(f"Dataset created: {output_filename}")
print(f"Total Sessions: {len(df_final)}")
print(f"Anomalous (Exfiltration) Sessions: {df_final['label'].sum()}")

# Show a preview
df_final.head()